In [1]:
# 代码参考了 https://github.com/facebookresearch/MetaICL

In [ ]:
%pip install numpy
%pip install pickle
%pip install transformers==4.28.1
%pip install torch==2.1.2

In [ ]:
import json
import os
import numpy as np
import pickle as pkl
import torch


In [3]:
!pwd

/cygdrive/f/llm-deploy/docs/chapter2/code/ICL


# 数据准备

In [ ]:
# 下载数据并解压到同级目录
# https://github.com/gyfffffff/llm-deploy/releases/download/icl_data/icl_data.zip

# 使用Linux 系统的同学也可以直接运行：
!wget https://github.com/gyfffffff/llm-deploy/releases/download/icl_data/icl_data.zip

In [7]:
# 读取数据
train_data_files = ["data/dream/dream_16_13_dev.jsonl", "data/wiki_qa/wiki_qa_16_13_dev.jsonl"]
train_data = []
for train_data_file in train_data_files:
    with open(train_data_file, "r") as f:
        data = []
        for line in f:
            data.append(json.loads(line))
    train_data.append(data)
train_data

[[{'task': 'dream',
   'input': "Based on the conversation, what is the most probable relationship between the speakers? [SEP] Heather: Ron, what are you doing? Ron: Ah, nothing. I'm just looking up some information on the Internet. Heather: Like what? Let me see. Ron: No, no, it's okay. I mean, you know ... Heather: Baldness? What are you looking that up for? [Well, you know ... ] I ... I mean, you're not that bad off. Ron: Ah, there you go. Bringing it up again! Heather: No. I mean it. You look great! Honestly, it's not that bad. Ron: Hey, I get enough of it from friends, and the people at work, and now from you! Heather: Well, maybe you could wear a toupee? I think you'd look great. Ron: Oh no. And have it slip off my head on to my date's dinner plate as I lean over to kiss her? Uh-uh. Heather: Well, have you ever thought about seeking medical advice? There are new advances in medicines that not only retard hair loss, but help regenerate new growth. Ron: Ah, I still don't give much 

In [ ]:
# 下载模型

# windows
%pip install -U "huggingface-hub[cli]"
!$env:HF_ENDPOINT = "https://hf-mirror.com"
!huggingface-cli download --resume-download openai-community/gpt2 --local-dir ../../models/GPT-2

# linux
%pip install -U "huggingface-hub[cli]"
!export HF_ENDPOINT=https://hf-mirror.com
!huggingface-cli download --resume-download openai-community/gpt2 --local-dir ../../models/GPT-2

�ļ�����Ŀ¼��������﷨����ȷ��


F:\llm-deploy\docs\chapter2\models\GPT-2


e:\Anaconda\envs\dl2\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]Downloading 'onnx/generation_config.json' to '..\..\models\GPT-2\.cache\huggingface\download\onnx\generation_config.json.30989900f4cebe446d28b25c9593001c7ef2853a.incomplete'
Download complete. Moving file to ..\..\models\GPT-2\onnx\generation_config.json
Download complete. Moving file to ..\..\models\GPT-2\onnx\tokenizer_config.json
Download complete. Moving file to ..\..\models\GPT-2\onnx\merges.txt
Download complete. Moving file to ..\..\models\GPT-2\onnx\special_tokens_map.json
Download complete. Moving file to ..\..\models\GPT-2\onnx\vocab.json

Fetching 26 files:  54%|█████▍    | 14/26 [00:03<00:02,  4.02it/s]Download complete. Moving file to ..\.

In [8]:
max_length=1024
max_length_per_example=256
model_path = "../../models/GPT-2"
batch_size = 4

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

def prepro_sentence_pair_single(ids1, ids2, max_length,
                                bos_token_id, eos_token_id,
                                allow_truncation=False):

    #if bos_token_id is not None:
    #    ids1 = [bos_token_id] + ids1
    #if eos_token_id is not None:
    #    ids2 = ids2 + [eos_token_id]
    if allow_truncation and len(ids1)+len(ids2) > max_length:
        ids1 = ids1[len(ids1)+len(ids2)-max_length:] # len = max_length-len(ids2)
        assert len(ids1)+len(ids2)==max_length

    n_mask = max_length-len(ids1)-len(ids2)
    assert n_mask>=0, (max_length, len(ids1), len(ids2))
    input_ids = ids1+ids2+[0 for _ in range(n_mask)]
    attention_mask = [1 for _ in ids1+ids2] + [0 for _ in range(n_mask)]
    token_type_ids = [0 for _ in ids1] + [1 for _ in ids2] + [0 for _ in range(n_mask)]
    return input_ids, attention_mask, token_type_ids



def _prepro_each_datapoint(dp, is_first=True, is_training=False, for_demonstrations=False):
    dp = dp.copy()

    no_label = np.all([option=="" for option in dp["options"]])
    no_input = dp["input"]==""
    if not is_first:
        dp["output"] = "\n\n\n" + dp["output"]
        if "options" in dp:
            dp["options"] = ["\n\n\n" + opt for opt in dp["options"]]
    if not no_input:
        if not no_label:
            dp["input"] = "\n" + dp["input"]

    input_tokens = tokenizer(dp["input"])["input_ids"]

    if is_training or for_demonstrations:
        output_tokens = tokenizer(dp["output"])["input_ids"]

        if "task" in dp:
            if len(input_tokens)>=max_length_per_example - 2 - len(output_tokens):
                if dp["task"].startswith("inst:") and len(input_tokens)<len(output_tokens):
                    output_tokens = output_tokens[:max_length_per_example - 2 - len(input_tokens)]
                else:
                    input_tokens = input_tokens[:max_length_per_example - 2 - len(output_tokens)]

        assert len(input_tokens)+len(output_tokens)+2<=max_length_per_example, \
            (dp.get("task", None), len(input_tokens), len(output_tokens), max_length_per_example)

        return output_tokens, input_tokens


    else:
        assert len(dp["options"])>=2, dp
        assert dp["output"] in dp["options"]
        option_tokens = [tokenizer(option)["input_ids"] for option in dp["options"]]
        option_length = np.max([len(option) for option in option_tokens])

        if len(input_tokens)>=max_length_per_example - 2 - option_length:
            input_tokens = input_tokens[:max_length_per_example - 2 - option_length]

        input_tokens = [input_tokens for _ in option_tokens]
        output_tokens = option_tokens
        option_tokens = [dp["options"].index(dp["output"])]

        return output_tokens, input_tokens, option_tokens


def _tensorize_for_training(train_data):
    for dp in train_data:
        assert type(dp)==dict, ("Each example should be a dictionary", dp)
        assert "input" in dp and "output" in dp, ("Training example should contain input and output", dp)

    # each datapoint: passage, question, options, output
    bos_token_id = tokenizer.bos_token_id
    eos_token_id = tokenizer.eos_token_id

    input_ids, attention_mask, token_type_ids = [], [], []


    for dp in train_data:
        inputs, outputs = _prepro_each_datapoint(
            dp, is_first=True, is_training=True)

        encoded = prepro_sentence_pair_single(
            inputs, outputs, max_length, bos_token_id, eos_token_id)

        input_ids.append(encoded[0])
        attention_mask.append(encoded[1])
        token_type_ids.append(encoded[2])

    return dict(input_ids=torch.LongTensor(input_ids),
                attention_mask=torch.LongTensor(attention_mask),
                token_type_ids=torch.LongTensor(token_type_ids))

# 数据转为tensor

def tensorize_for_training(train_data, is_training=True):
    inputs = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

    for in_ in train_data:
        out = _tensorize_for_training(in_)
        for key in ["input_ids", "attention_mask", "token_type_ids"]:
            inputs[key] += out[key].numpy().tolist()

    N = len(inputs["input_ids"])
    indices = np.random.permutation(range(N))
    for k, v in inputs.items():
        inputs[k] = np.array(v)[indices]

    with open("data/preprocessed_data.kpl", "wb") as f:
        pkl.dump({k:v for k, v in inputs.items()}, f)
    print("Finish saving preprocessed data ...")

    for k, v in inputs.items():
        inputs[k] = torch.LongTensor(v)
    shape = inputs["input_ids"].shape
    for v in inputs.values():
        assert v.shape==shape
    if "labels" in inputs:
        dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], inputs["token_type_ids"], inputs["labels"])
    else:
        dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], inputs["token_type_ids"])
    if is_training:
        sampler=RandomSampler(dataset)
    else:
        sampler=SequentialSampler(dataset)
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)
    return dataloader
dataloader = tensorize_for_training(train_data)


Finish saving preprocessed data ...


# 训练模型

In [11]:
import logging

# 指定日志路径

out_dir = "output"
log_file = f"{out_dir}/log.txt"
handlers = [logging.StreamHandler()]
handlers.append(logging.FileHandler(log_file))
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO,
                    handlers=handlers)
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)
logger = logging.getLogger(__name__)
logger.info(out_dir)

os.makedirs(out_dir, exist_ok=True)


11/30/2024 09:42:40 - INFO - __main__ - output


In [12]:
import torch

device = torch.device("cpu")
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_path)
model.to(device)
from transformers import Adafactor, AdamW, get_linear_schedule_with_warmup

save_period = 10
log_period = 5
batch_size = 4
num_training_steps = 1000

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters,
                    lr=1e-05,
                    eps=1e-08)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
        
model.train()

def run_model(input_ids, attention_mask, token_type_ids, labels=None):
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits[..., :-1, :].contiguous()

        if labels is None:
            labels = input_ids
        labels = labels[..., 1:].contiguous()
        label_mask = token_type_ids[..., 1:].contiguous()

        loss_fct = torch.nn.CrossEntropyLoss(reduction="none")
        losses = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1)) # [batch_size, length]

        losses = losses.view(logits.size(0), logits.size(1)) * label_mask
        return torch.sum(losses, axis=1) / torch.sum(label_mask, axis=1)

def save(step):
        model_state_dict = {key[7:] if key.startswith("module.") else key: value.cpu()
                        for key, value in model.state_dict().items()}
        torch.save(model_state_dict, os.path.join(out_dir, "model-{}.pt".format(step)))
        logger.info("Saving model parameters at step=%d" % step)

def do_train(data, batch_size, num_training_steps, save_period, log_period, gradient_accumulation_steps=1, max_grad_norm=1.0):
        n_trainable_params = len([param for param in model.parameters() if param.requires_grad])
        n_gpus = 1
        global_step = 0
        train_losses = []
        best_accuracy = -1
        stop_training=False

        for epoch in range(num_training_steps):
                for batch in dataloader:
                        global_step += 1

                        input_ids=batch[0].to(device)
                        attention_mask=batch[1].to(device)
                        token_type_ids=batch[2].to(device)
                        if len(batch)==3:
                                labels=None
                        else:
                                labels=batch[3].to(device)

                        loss = run_model(input_ids, attention_mask, token_type_ids=token_type_ids, labels=labels)
                        loss = loss.mean()

                        train_losses.append(loss.detach().cpu())

                        loss.backward()

                        if global_step % gradient_accumulation_steps == 0:
                                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

                                optimizer.step()    # We have accumulated enought gradients
                                if scheduler is not None:
                                        scheduler.step()
                                        model.zero_grad()

                        if global_step % log_period == 0:
                                logger.info("global step %d\t train loss %.2f" % (global_step, np.mean(train_losses)))
                                train_losses = []

                        if global_step % save_period == 0:
                                save(global_step)

                        if global_step==num_training_steps:
                                break

                if global_step==num_training_steps:
                        break

        logger.info("Finish training")
do_train(data, batch_size, num_training_steps, save_period, log_period=log_period)

e:\Anaconda\envs\dl2\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:98                                                                                   │
│                                                                                                  │
│   95 │   │   │   │   │   │   break                                                               │
│   96 │   │                                                                                       │
│   97 │   │   logger.info("Finish training")                                                      │
│ ❱ 98 do_train(data, batch_size, num_training_steps, save_period, log_period=log_period)          │
│   99                                                                                             │
│                                                                                                  │
│ in do_train:69                                                                                   │
│                                                                                                  │
│   66 │   │   │   │   │   │   else:                                                               │
│   67 │   │   │   │   │   │   │   │   labels=batch[3].to(device)                                  │
│   68 │   │   │   │   │   │                                                                       │
│ ❱ 69 │   │   │   │   │   │   loss = run_model(input_ids, attention_mask, token_type_ids=token    │
│   70 │   │   │   │   │   │   loss = loss.mean()                                                  │
│   71 │   │   │   │   │   │                                                                       │
│   72 │   │   │   │   │   │   train_losses.append(loss.detach().cpu())                            │
│                                                                                                  │
│ in run_model:29                                                                                  │
│                                                                                                  │
│   26 model.train()                                                                               │
│   27                                                                                             │
│   28 def run_model(input_ids, attention_mask, token_type_ids, labels=None):                      │
│ ❱ 29 │   │   outputs = model(input_ids=input_ids, attention_mask=attention_mask)                 │
│   30 │   │   logits = outputs.logits[..., :-1, :].contiguous()                                   │
│   31 │   │                                                                                       │
│   32 │   │   if labels is None:                                                                  │
│                                                                                                  │
│ e:\Anaconda\envs\dl2\lib\site-packages\torch\nn\modules\module.py:1518 in _wrapped_call_impl     │
│                                                                                                  │
│   1515 │   │   if self._compiled_call_impl is not None:                                          │
│   1516 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]        │
│   1517 │   │   else:                                                                             │
│ ❱ 1518 │   │   │   return self._call_impl(*args, **kwargs)                                       │
│   1519 │                                                                                         │
│   1520 │   def _call_impl(self, *args, **kwargs):                                                │
│   1521 │   │   forward_call = (self._slow_forward if torch._C._get_tracing_state() else self.fo  │
│                                                                                                  │
│ e:\Anaconda\envs\dl2\lib\site-packages\torch\nn\modules\mod